In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date,datetime, timedelta
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier

In [2]:
speed = pd.read_csv('train.csv',index_col='date',parse_dates=['date'],dayfirst = True)
speed2 = pd.read_csv('test.csv',index_col='date',parse_dates=['date'],dayfirst = True)

In [3]:
speed = speed[['speed']]

In [4]:
speed_train1, speed_test1 = train_test_split(speed,test_size = 0.2,random_state = 2020)


In [5]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek

    df['night'] = ((df['date'].dt.hour<=6)|(df['date'].dt.hour>18)).astype(int)
    df['day'] = (df['night'] == 0).astype(int)
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year 
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','night','day','quarter','year','dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [6]:
X_train1, y_train1 = create_features(speed_train1, label='speed')
X_test1, y_test1 = create_features(speed_test1, label='speed')


<ipython-input-5-659fa3d3609e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df.index
<ipython-input-5-659fa3d3609e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['date'].dt.hour
<ipython-input-5-659fa3d3609e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [7]:
reg1 = xgb.XGBRegressor(n_estimators=5000,learning_rate=0.07,max_depth=7,gamma =0.1,subsample = 0.8)

reg1.fit(X_train1, y_train1,
        early_stopping_rounds = 300,
        eval_set=[ (X_test1, y_test1)],
        verbose=True) # Change verbose to True if you want to see it train

[0]	validation_0-rmse:32.46008
Will train until validation_0-rmse hasn't improved in 300 rounds.
[1]	validation_0-rmse:30.23629
[2]	validation_0-rmse:28.17248
[3]	validation_0-rmse:26.25670
[4]	validation_0-rmse:24.47753
[5]	validation_0-rmse:22.82483
[6]	validation_0-rmse:21.28958
[7]	validation_0-rmse:19.86303
[8]	validation_0-rmse:18.54110
[9]	validation_0-rmse:17.31835
[10]	validation_0-rmse:16.18469
[11]	validation_0-rmse:15.13399
[12]	validation_0-rmse:14.15940
[13]	validation_0-rmse:13.26334
[14]	validation_0-rmse:12.43479
[15]	validation_0-rmse:11.67299
[16]	validation_0-rmse:10.96364
[17]	validation_0-rmse:10.30740
[18]	validation_0-rmse:9.70720
[19]	validation_0-rmse:9.15677
[20]	validation_0-rmse:8.65064
[21]	validation_0-rmse:8.18533
[22]	validation_0-rmse:7.75767
[23]	validation_0-rmse:7.36987
[24]	validation_0-rmse:7.01231
[25]	validation_0-rmse:6.69071
[26]	validation_0-rmse:6.39702
[27]	validation_0-rmse:6.12791
[28]	validation_0-rmse:5.88436
[29]	validation_0-rmse:5.66

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.07, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=5000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [8]:
### 性能度量
y_pred1 = reg1.predict(X_test1)
from sklearn.metrics import accuracy_score,mean_squared_error  # 准确率
accuracy = mean_squared_error(y_test1,y_pred1)
print("mse: %.2f%%"  %(accuracy))

mse: 10.87%


In [9]:
X_test, y_test = create_features(speed2, label='id')
X_test

,hour,dayofweek,night,day,quarter,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,
2018-01-01 02:00:00,2,0,1,0,1,2018,1,1,1
2018-01-01 05:00:00,5,0,1,0,1,2018,1,1,1
2018-01-01 07:00:00,7,0,0,1,1,2018,1,1,1
2018-01-01 08:00:00,8,0,0,1,1,2018,1,1,1
2018-01-01 10:00:00,10,0,0,1,1,2018,1,1,1
...,...,...,...,...,...,...,...,...,...
2018-12-31 17:00:00,17,0,0,1,4,2018,365,31,1
2018-12-31 19:00:00,19,0,1,0,4,2018,365,31,1
2018-12-31 21:00:00,21,0,1,0,4,2018,365,31,1


In [10]:
#df.sort_index(ascending=False)
X_test.head(10)

,hour,dayofweek,night,day,quarter,year,dayofyear,dayofmonth,weekofyear
date,,,,,,,,,
2018-01-01 02:00:00,2,0,1,0,1,2018,1,1,1
2018-01-01 05:00:00,5,0,1,0,1,2018,1,1,1
2018-01-01 07:00:00,7,0,0,1,1,2018,1,1,1
2018-01-01 08:00:00,8,0,0,1,1,2018,1,1,1
2018-01-01 10:00:00,10,0,0,1,1,2018,1,1,1
2018-01-01 16:00:00,16,0,0,1,1,2018,1,1,1
2018-01-01 18:00:00,18,0,0,1,1,2018,1,1,1
2018-01-02 01:00:00,1,1,1,0,1,2018,2,2,1
2018-01-02 02:00:00,2,1,1,0,1,2018,2,2,1


In [11]:
speed_testing = reg1.predict(X_test)

#pjme_test['MW_Prediction'] = reg.predict(X_test)

In [12]:
speed_testing = pd.DataFrame(speed_testing)
speed_testing.head(5)

,0
0,48.019314
1,47.875473
2,38.483986
3,30.048992
4,41.216118


In [13]:
Submission= pd.read_csv('sampleSubmission.csv')
Submission

,id,speed
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
3499,3499,0
3500,3500,0
3501,3501,0
3502,3502,0


In [14]:
Submission['speed'] = speed_testing
Submission

,id,speed
0,0,48.019314
1,1,47.875473
2,2,38.483986
3,3,30.048992
4,4,41.216118
...,...,...
3499,3499,14.735497
3500,3500,21.381052
3501,3501,45.817318
3502,3502,39.650616


In [15]:
Submission.to_csv('xgboost_submission.csv',index=False,header = True)